In [1]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tensorflow.python.framework import ops
import cv2
import os
import numpy as np
from IPython.display import clear_output
import tkinter
import PIL.Image, PIL.ImageTk
import time
import threading
import tweepy

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
class MyVideoCapture:
    def __init__(self, video_source=0):
         # Open the video source
        self.vid = cv2.VideoCapture(video_source)
        if not self.vid.isOpened():
            raise ValueError("Unable to open video source", video_source)
 
         # Get video source width and height
        self.width = self.vid.get(cv2.CAP_PROP_FRAME_WIDTH)
        self.height = self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
 
    def get_frame(self):
        if self.vid.isOpened():
            ret, frame = self.vid.read()
            if ret:
                # Return a boolean success flag and the current frame converted to BGR
                return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            else:
                return (ret, None)
        else:
            return (ret, None)
 
     # Release the video source when the object is destroyed
    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()

In [3]:
class face_reg:
    def __init__(self):
        self.make_model()
        
    def face_detection(self,img):
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        face_cordinates = face_cascade.detectMultiScale(gray, 1.3, 5)
        faces = []
        if len(face_cordinates)>0:
            no_of_faces = face_cordinates.shape[0]
        else:
            no_of_faces = 0
        for i in range(no_of_faces):
            faces.append(img[face_cordinates[i][1]:face_cordinates[i][1]+face_cordinates[i][3],face_cordinates[i][0]:face_cordinates[i][0]+face_cordinates[i][2],:])
        return faces
    
    def get_data(self,name,cap):
        i = 1
        try:
            path = os.curdir + "/data/" + name
            os.mkdir(path) 
        except FileExistsError:
            print("Data with the name " , name ,  " already exists")
            return
        while(i<100):
            ret,frame = cap.get_frame()
            faces = self.face_detection(frame)
            if len(faces)>0:
                faces[0] = cv2.cvtColor(faces[0], cv2.COLOR_BGR2GRAY)
                cv2.imwrite(path + "/" + str(i) + ".jpg",faces[0])
                i = i+1
        self.make_model()
        return

    def make_model(self):
        names = os.listdir("Data")
        train_images = np.empty((0,50,50,1), dtype=np.float64)
        temp_array = np.empty((50,50,1), dtype=np.float64)
        train_labels = np.empty((0,len(names)),dtype=np.uint8)
        label_array = np.empty((len(names)), dtype=np.uint8)
        no_data = 0
        for i in range(len(names)):
            images = os.listdir("Data/" + names[i])
            label_array = [0 for i in range(len(names))]
            label_array[i] = 1 
            for j in range(len(images)):
                clear_output(True)
                print(str(no_data))
                img = cv2.imread("Data/" + names[i] + "/" + images[j], 1)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img,(50,50))
                temp_array[:,:,0] = img
                train_images = np.append(train_images,np.array([temp_array]),axis=0)
                train_labels = np.append(train_labels,np.array([label_array]),axis=0)
                no_data = no_data + 1
        train_images = train_images
        
        ops.reset_default_graph()
        convnet = input_data(shape=[50,50,1])
        convnet = conv_2d(convnet, 32, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)
        convnet = conv_2d(convnet, 64, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)
        convnet = conv_2d(convnet, 128, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)
        convnet = conv_2d(convnet, 64, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)
        convnet = conv_2d(convnet, 32, 5, activation='relu')
        convnet = max_pool_2d(convnet, 5)

        convnet = fully_connected(convnet, 1024, activation='relu')
        convnet = dropout(convnet, 0.8)
        convnet = fully_connected(convnet, len(names), activation='softmax')
        convnet = regression(convnet, optimizer='adam', learning_rate = 0.001, loss='categorical_crossentropy')
        self.model = tflearn.DNN(convnet, tensorboard_verbose=1)
        self.history = self.model.fit(train_images, train_labels, n_epoch=15, show_metric = True, run_id="FRS" )
        return
    
    def recognise(self,cap):
        names = os.listdir("Data")
        temp_array = np.empty((50,50,1), dtype=np.float64)
        i = 0
        j = 0
        while(i<200):
            ret,frame = cap.get_frame()
            faces = self.face_detection(frame)
            if len(faces)>0:
                img = cv2.cvtColor(faces[0], cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img,(50,50))
                temp_array[:,:,0] = img
                self.predictions = self.model.predict([temp_array])
                if np.max(self.predictions[0])>0.8:
                    j = j+1
                    if j>=3:
                        return names[np.argmax(self.predictions[0])]
            i = i+1
            cv2.waitKey(20)
        return "Not recognised"

In [4]:
class App:
    def __init__(self, window, window_title, video_source=0):
        self.window = window
        self.window.title(window_title)
        self.video_source = video_source
        
        self.pwthaut = ["Anas"]
 
        # Open video source (by default this will try to open the computer webcam)
        self.vid = MyVideoCapture(self.video_source)
        
        self.freg = face_reg()
        
        self.t1 = threading.Thread(target=self.aut)
        self.t2 = threading.Thread(target=self.gd)
        
        self.autst = "Not Authorised"
 
        # Create a canvas that can fit the above video source size
        self.canvas = tkinter.Canvas(window, width = self.vid.width, height = self.vid.height)
        self.canvas.pack()
        
        self.output_field=tkinter.Label(window, text = "Hello")
        self.output_field.pack(anchor=tkinter.CENTER, expand=True)
 
        # Button 1
        self.btn_snapshot=tkinter.Button(window, text="Authorise", width=50, command=self.t1.start)
        self.btn_snapshot.pack(anchor=tkinter.CENTER, expand=True)
        
        tkinter.Label(self.window,text="Enter Name").pack(anchor=tkinter.CENTER, expand=True)
        self.input_field=tkinter.Entry(window)
        self.input_field.pack(anchor=tkinter.CENTER, expand=True)
        
        # Button 2
        self.btn_snapshot=tkinter.Button(window, text="Get Data", width=50, command=self.t2.start)
        self.btn_snapshot.pack(anchor=tkinter.CENTER, expand=True)
        
        self.output_field_end=tkinter.Label(window)
        self.output_field_end.pack(anchor=tkinter.CENTER, expand=True)
        
         # After it is called once, the update method will be automatically called every delay milliseconds
        self.delay = 10
        self.update()
 
        self.window.mainloop()
    
    def aut(self):
        self.acc = self.freg.recognise(self.vid)
        if self.acc in self.pwthaut:
            self.autst = "Authorised"
            self.output_field.configure(text = "Authorised")
        else:
            self.output_field.configure(text = "Not Authorised")
        
    def gd(self):
        self.freg.get_data(self.input_field.get(),self.vid)
        self.output_field_end.configure(text = "Data Aquired")
        
    def tweet(self):
        #my twitter account's key, if you use this keys, tweets will be posted in my account
        #So make a twitter account, apply for developer profile... then you will get the keys
        consumer_key = "g8CXca2B3q9dq2DsVWi7L3xEZ"
        consumer_secret = "qCyMR2D4BEiI7Uuhmv1rredpj2lhQxCL4hHPa3iliWlle1nMv7"
        access_token = "1410950779913990144-husWMAhCKJskjfVwug1Wb9PZ32qZPI"
        access_token_secret = "wTVr17ilI6XkgesNwUaazYtKQB4vy0E665Zjve1gY8qer"

        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth)

        api.update_status(status = self.input_field_tw.get("1.0",tkinter.END))
        
        timenow = time.ctime(time.time())
        self.output_field_cnfm1.configure(text = "Tweeted by "+self.acc)
        self.output_field_cnfm2.configure(text = "At time "+timenow)
        self.fout = open("Tweet.txt", 'w')
        self.fout.write(self.input_field_tw.get("1.0",tkinter.END))
        self.fout.close()
        self.fout = open("Log.txt", 'a')
        self.fout.write(self.acc+" "+timenow+"\n")
        self.fout.close()
        
        
    def newupdate(self):
        for widgets in self.window.winfo_children():
             widgets.pack_forget()
        
        tkinter.Label(self.window,text="Enter Tweet").pack()
        self.input_field_tw=tkinter.Text(self.window,font= "Helvetica 15 bold",height=10,width=80)
        self.input_field_tw.pack(anchor=tkinter.CENTER)
        
        #tkinter.Label(self.window,text="Enter Time").place(x=100,y=100)
        #self.input_field_tm=tkinter.Entry(self.window)
        #self.input_field_tm.place(anchor=tkinter.CENTER,x = 300,y = 100)
        
        self.btn_snapshot=tkinter.Button(self.window, text="Confirm", width=50, command=self.tweet)
        self.btn_snapshot.pack(anchor=tkinter.CENTER)
        
        self.output_field_cnfm1=tkinter.Label(self.window)
        self.output_field_cnfm1.pack(anchor=tkinter.CENTER)
        
        self.output_field_cnfm2=tkinter.Label(self.window)
        self.output_field_cnfm2.pack(anchor=tkinter.CENTER)
        
        self.window.mainloop()
            

    def update(self):
         # Get a frame from the video source
        ret, frame = self.vid.get_frame()
 
        if ret and self.autst == "Not Authorised":
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
            self.canvas.create_image(0, 0, image = self.photo, anchor = tkinter.NW)
        if self.autst == "Authorised":
            self.newupdate()
 
        self.window.after(self.delay, self.update)
    
 
 # Create a window and pass it to the Application object
App(tkinter.Tk(), "Twitter BOT")


Training Step: 44  | total loss: 0.56585 | time: 0.192s
| Adam | epoch: 015 | loss: 0.56585 - acc: 0.8337 -- iter: 128/183
Training Step: 45  | total loss: 0.47959 | time: 0.290s
| Adam | epoch: 015 | loss: 0.47959 - acc: 0.8619 -- iter: 183/183
--
